![](imgs/kodolamaczlogo.png)

# Przetwarzanie Big Data z użyciem Apache Spark

Autor notebooka: Jakub Nowacki.

## Uczenie Maszynowe (Machine Learning) na Spark

Spark ML zawiera wiele algorytmów uczenia maszynowego, które się dobrze działają w sposób rozproszony i się skalują, w tym:

* regresja liniowa,
* regresja logistyczna,
* algorytm random forest,
* algorytm centroidów (k-means).

To API wykorzystuje RDD i wymaga nieco pracy z przydotowaniem danych.

In [1]:
import pyspark

spark = pyspark.sql.SparkSession.builder \
    .appName("SparkMLlib") \
    .getOrCreate()
sc = spark.sparkContext

In [2]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD
from operator import add

In [3]:
# Ładujemy dane z pliku CSV.
# Szczegółowy opis danych jest dostępny w pliku README
rdd = sc.textFile("data/Bike-Sharing-Dataset/day.csv")

In [4]:
rdd.first()

'instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt'

In [5]:
all_columns = rdd.first().split(",")
all_columns

['instant',
 'dteday',
 'season',
 'yr',
 'mnth',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed',
 'casual',
 'registered',
 'cnt']

In [7]:
x_cols = ['yr', 'workingday', 'weathersit', 'temp', 'hum', 'windspeed']
y_col = 'casual'
x_inds = [i for i, col in enumerate(all_columns) if col in x_cols]
y_ind = all_columns.index(y_col)
x_cols, x_inds, y_col, y_ind

(['yr', 'workingday', 'weathersit', 'temp', 'hum', 'windspeed'],
 [3, 7, 8, 9, 11, 12],
 'casual',
 13)

In [20]:
import math

In [53]:
lps = rdd \
  .filter(lambda line: not line.startswith("instant")) \
  .map(lambda line: line.split(",")) \
  .map(lambda values: LabeledPoint(math.log10(float(values[y_ind])), [float(values[i]) for i in x_inds])) \
  .cache()

In [54]:
lps.take(5)

[LabeledPoint(2.519827993775719, [0.0,0.0,2.0,0.344167,0.805833,0.160446]),
 LabeledPoint(2.1172712956557644, [0.0,0.0,2.0,0.363478,0.696087,0.248539]),
 LabeledPoint(2.0791812460476247, [0.0,1.0,1.0,0.196364,0.437273,0.248309]),
 LabeledPoint(2.03342375548695, [0.0,1.0,1.0,0.2,0.590435,0.160296]),
 LabeledPoint(1.9138138523837167, [0.0,1.0,1.0,0.226957,0.436957,0.1869])]

In [55]:
# zobaczmy co znajduje się w LabelPoint
p = lps.first()

In [56]:
p.label

2.519827993775719

In [57]:
p.features

DenseVector([0.0, 0.0, 2.0, 0.3442, 0.8058, 0.1604])

In [58]:
# tworzenie regresora i uczenie na danych
lrspark = LinearRegressionWithSGD.train(lps, iterations=100, intercept=True)

/home/kodolamacz/spark/python/pyspark/mllib/regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")


In [59]:
# predykcja
lrspark.predict([0.0,0.0,2.0,0.344167,0.805833,0.160446])

2.6218774814630832

In [60]:
# prawdziwe odpowiedzi i predykcja
y_ypred = lps.map(lambda x: (x.label, lrspark.predict(x.features))).cache()

In [61]:
y_ypred.take(5)

[(2.519827993775719, 2.6218774814630832),
 (2.1172712956557644, 2.6050074724317289),
 (2.0791812460476247, 2.1897058461256482),
 (2.03342375548695, 2.2516875734504036),
 (1.9138138523837167, 2.2151790578006851)]

In [62]:
# policzmy R^2 ręcznie

mean_y = y_ypred.map(lambda x: x[0]).mean()

variance_y = y_ypred.map(lambda x: x[0]).variance()

residual_variance_y = y_ypred \
  .map(lambda x: (x[0] - x[1])**2) \
  .reduce(add) / y_ypred.count()

R2 = 1 - residual_variance_y/variance_y

print("Mean: {}, Variance: {}, Residual Variance: {}".format(mean_y, variance_y, residual_variance_y))
print("R^2 is: {:.3f}".format(R2))

Mean: 2.7568155886472994, Variance: 0.19532857807621223, Residual Variance: 0.07397992619410722
R^2 is: 0.621


In [65]:
[10**p[1] for p in y_ypred.take(10)]

[418.67543619808464,
 402.72396349808105,
 154.77679384111997,
 178.52028580106682,
 164.12663209690243,
 163.77026089420562,
 109.40290057904967,
 206.6729364757542,
 263.29389727318977,
 143.49716676341191]

### Zadanie

* Użyj skali logarytmicznej dla `casual`.
* ★ Przeprowadź kroswalidację (podpowiedź: `lps.randomSplit([0.75, 0.25])`).

In [68]:
train, validate, test = lps.randomSplit([0.6, 0.2, 0.2])